In [1]:
import os

from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
files_list = os.listdir('.\data')

In [3]:
files_list[:10]

['20200801_Data.mdf',
 '20200801_Log.ldf',
 '20200808_20200810_Data.mdf',
 '20200808_20200810_Log.ldf',
 '20200811_20200814_Data.mdf',
 '20200811_20200814_Log.ldf',
 '20200817_20200821_Data.mdf',
 '20200817_20200821_Log.ldf',
 '20200822_20200828-2_Data.mdf',
 '20200822_20200828-2_Log.ldf']

In [4]:
len(files_list)

214

Create a script for attaching mdf files

In [5]:
add_db_query = ""
for i in range(0, len(files_list), 2):
    data_file =  files_list[i]
    log_file =  files_list[i+1]
    add_db_query = add_db_query + f"""
                exec sp_attach_db @dbname = N'{data_file.split(".")[0]}', 
                @filename1=N'D:\My\Python\\Neo Result Viewer\data\{data_file}',
                @filename2=N'D:\My\Python\\Neo Result Viewer\data\{log_file}';
                """

In [6]:
# Establish database connection
# Params
database_type = 'mssql'
database_connector = 'pyodbc'
login = 'sa' 
password = 'Nh3002'
server = 'LAPTOP-U3H8SBSH'
database = 'master' 
driver = 'SQL+Server'

engine = create_engine(f"{database_type}+{database_connector}://{login}:{password}@{server}/{database}?driver={driver}")

In [7]:
columns = "LogID, AssayName, SampleNumber, PlateName, AssayType, Result1, Result2, Result5, Pattern1, Pattern2, RawDataValue0, CreatedDate, WellID, Tags, InstrumentDateTime"
table = "HistorySample"
dbnames = [file.split(".")[0] for file in files_list[::2]]

for index, dbname in enumerate(dbnames):
    if index == 0:
        query = (f"SELECT {columns}\n"
                 f"FROM [{dbname}].dbo.[{table}]\n")
    else:
        query = query + "UNION ALL\n" + f"SELECT {columns}\n" + f"FROM [{dbname}].dbo.[{table}]\n"

In [8]:
#print(query)

In [9]:
df = pd.read_sql_query(query, engine)

In [12]:
df.to_csv("raw_data.csv", index=False)